# Problem :

Geologists sometimes divide an area of land into different regions based on where rainfall flows down to. These regions are called drainage basins.

Given an elevation map (a 2-dimensional array of altitudes), label the map such that locations in the same drainage basin have the same label, subject to the following rules.

- From each cell, water flows down to at most one of its 4 neighboring cells.
- For each cell, if none of its 4 neighboring cells has a lower altitude than the current cell's, then the water does not flow, and the current cell is called a sink.
- Otherwise, water flows from the current cell to the neighbor with the lowest altitude.
- In case of a tie, water will choose the first direction with the lowest altitude from this list: North, West, East, South.

Every cell that drains directly or indirectly to the same sink is part of the same drainage basin. Each basin is labeled by a unique lower-case letter, in such a way that, when the rows of the map are concatenated from top to bottom, the resulting string is lexicographically smallest. (In particular, the basin of the most North-Western cell is always labeled 'a'.)

## Input

The first line of the input file will contain the number of maps, T. T maps will follow, each starting with two integers on a line -- H and W -- the height and width of the map, in cells. The next H lines will each contain a row of the map, from north to south, each containing W integers, from west to east, specifying the altitudes of the cells.

## Output

For each test case, output 1+H lines. The first line must be of the form

Case #X:
where X is the test case number, starting from 1. The next H lines must list the basin labels for each of the cells, in the same order as they appear in the input.

In [1]:
import numpy as np

### Implementation

In [2]:
def getLine():
    # For easily reading one line
    return list(map(int, input().split(' ')))


def getInput():
    # For easily reading the input as in the description
    nbMaps = int(input())
    elevationMaps = []
    
    for i in range(nbMaps):
        # Reading the ith map
        size = getLine()
        elevationMap = np.array([np.array(getLine()) for ii in range(size[0])])
        elevationMaps.append(elevationMap)
    return elevationMaps


def getOrderedNeighbors(i, j, iMax, jMax):
    neighbors = []
    if i - 1 >= 0:   neighbors.append((i - 1, j))
    if j - 1 >= 0:   neighbors.append((i, j - 1))
    if j + 1 < jMax: neighbors.append((i, j + 1))
    if i + 1 < iMax: neighbors.append((i + 1, j))
    return neighbors


def getCorrespondingSink(elevationMap, visitedPoints, i, j):
    # Initialization
    newI, newJ = i, j
    oldI, oldJ = None, None
    iMax, jMax = elevationMap.shape
    path = [(newI, newJ)]
    
    # Looping through the points following the water flow
    # When we arrive at the sink there would be no  lower
    # points so the old point is the new point!
    while (oldI, oldJ) != (newI, newJ):
        oldI, oldJ = newI, newJ
        for ii, jj in getOrderedNeighbors(newI, newJ, iMax, jMax):
            if elevationMap[ii,jj] < elevationMap[newI,newJ]:
                newI, newJ = ii, jj
        # We found a lower point, so we add it to the path and go downhill
        # Last two point maybe duplicated but it is not a big prblem ...
        path.append((newI, newJ))

        # If we already visited this point we return, otherwise we continue downhill
        if (newI, newJ) in visitedPoints:
            return path
    return path


def getMapLabeling(elevationMap):
    # Initialization
    newLabel = 97
    mapLabeling = np.ndarray(elevationMap.shape)
    
    # Stores sinks labels and
    # Serves also as cache ..
    visitedPoints = {}
    
    # For each point we find its corresponding sink
    # and we give it the same label.
    for i in range(elevationMap.shape[0]):
        for j in range(elevationMap.shape[1]):
            if (i, j) in visitedPoints:
                # This point was visited so we already know its label
                mapLabeling[i, j] = visitedPoints[i, j]                
            else:
                # This point was not visited before so we get its corresponding sink
                path = getCorrespondingSink(elevationMap, visitedPoints, i, j)
                sink = path[-1]
                
                # If the sink was not visited we give it a new label
                if not sink in visitedPoints:
                    visitedPoints[sink] = newLabel
                    newLabel += 1
                # And for each point in the path to the sink we must update the label 
                for point in path:
                    mapLabeling[point]   = visitedPoints[sink]
                    visitedPoints[point] = visitedPoints[sink]
    return mapLabeling


def printMapLabeling(mapLabeling, i):
    # Printing the solution as in description ...
    print('Case #{}:'.format(i))
    for i in range(mapLabeling.shape[0]):
        print(' '.join(list(map(lambda x : chr(int(x)), mapLabeling[i]))))

###  Solving the problem

In [3]:
for i, elevationMap in enumerate(getInput()):
    labeling = getMapLabeling(elevationMap)
    printMapLabeling(labeling, i + 1)

1
5 11
5 5 5 5 5 9 7 7 7 7 7
5 4 4 4 5 9 7 6 6 6 7
5 4 3 4 5 9 7 6 5 6 7
5 4 4 4 5 9 7 6 6 6 7
5 5 5 5 5 9 7 7 7 7 7
Case #1:
a b c d e e f g h i j
b b c d d d g g h i i
c c c c c c h h h h h
k k c l l l m m h n n
o k c l p p q m h n r


In [4]:
#5
#3 3
#9 6 3
#5 9 6
#3 5 9
#1 10
#0 1 2 3 4 5 6 7 8 7
#2 3
#7 6 7
#7 6 7
#5 5
#1 2 3 4 5
#2 9 3 9 6
#3 3 0 8 7
#4 9 8 9 8
#5 6 7 8 9
#2 13
#8 8 8 8 8 8 8 8 8 8 8 8 8
#8 8 8 8 8 8 8 8 8 8 8 8 8
#Case #1:
#a b b
#a a b
#a a a
#Case #2:
#a a a a a a a a a b
#Case #3:
#a a a
#b b b
#Case #4:
#a a a a a
#a a b b a
#a b b b a
#a b b b a
#a a a a a
#Case #5:
#a b c d e f g h i j k l m
#n o p q r s t u v w x y z